<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> SMA - Drug, Temporal, Order and Negation pattern extraction </h1><br>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Sruthi S, Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>19th April 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>7th Jun 2018</time>
        </div>
    </div>
</div>

# Objective:

The purpose of this notebook is to do pattern extraction for all sentences, which were classified to be containing a treatment pattern in the previous step. The output of this notebook is a dataframe with drug, order, negation words indices and pattern along with temporal words indices, words and converted date from temporal words. 

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import collections
import os
import tempDrugOrderNegTag as tag
#sutime is used for temporal words extraction and conversion
from sutime import SUTime
from dateutil import parser

#dateutil is used to get the absolute date given a reference date and delta days/month/years
from dateutil.rrule import rrule, MONTHLY, DAILY, WEEKLY
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
jar_files = 'sutime/jars/'
sutime = SUTime(jars=jar_files,jvm_started= False , mark_time_ranges= True , include_range= True )
sutime = SUTime(jars=jar_files,jvm_started= True , mark_time_ranges= True , include_range= True )

['lucene-core-4.10.3.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar', 'lucene-queryparser-4.10.3.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar', 'lucene-queryparser-4.10.3.jar', 'stanford-corenlp-3.9.1.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar', 'lucene-queryparser-4.10.3.jar', 'stanford-corenlp-3.9.1.jar', 'protobuf-java-3.2.0.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar', 'lucene-queryparser-4.10.3.jar', 'stanford-corenlp-3.9.1.jar', 'protobuf-java-3.2.0.jar', 'slf4j-simple.jar']
['lucene-core-4.10.3.jar', 'javax.servlet-api-3.0.1.jar', 'jollyday-0.4.9-sources.jar', 'lucene-queryparser-4.10.3.jar', 'stanford-corenlp-3.9.1.jar', 'protobuf-java-3.2.0.jar', '

In [3]:
global pattern_dict #Keys are indices and values are patterns like ['D','O','N']
global word_dict #Keys are indices and values are words like ['Ipilimumab','after','not']

# Will update the pattern_dict and word_dict for drug, order and neg tags when called
def get_index(typeString,replaceString,words_list,text):
    global pattern_dict
    global word_dict
    words = []
    
    # Replace all drugs/order/neg words by a replace string
    for i in range(0,len(words_list)):
        words.append(words_list[i]) 
        text = re.sub(words_list[i],replaceString ,text)
        
    z = text
    a = nltk.word_tokenize(z)
    # Since our replace string has space on both ends to handle few exceptions
    replaceString = replaceString.split(' ')[1]
    ind = [index for index, value in enumerate(a) if value == replaceString]
    count = 0
    # Updating the pattern_dict and word_dict
    for i in ind:
        pattern_dict[i] = typeString
        word_dict[i] = words[count]
        count +=1

In [4]:
# Returns drug, order, neg words indices and pattern along with temporal words indices, words, converted 
# temporal words
def patternExtract(text, postDate):
    global pattern_dict
    global word_dict
    pattern_dict = {}
    word_dict = {}
    
    #temp_list is a list of dict from sutime package
    temp_list = sutime.parse(text , reference_date= postDate)
    
    Temporal = [] #Temporal words detected
    Temp_pattern = [] #Encoded pattern like T0, T1, etc..
    Temp_conversion  = [] #Converted temporal words
    Temp_index  = [] #Index of temporal words
    
    # append the temporal word, pattern and its conversion to their respective list for every detected temporal word
    # temp_list[i] is a dict
    
    for i in range(len(temp_list)):
        Temporal.append(temp_list[i]['text'].encode('utf-8'))
        Temp_pattern.append('T'+re.escape(str(i)))
        try:
            Temp_conversion.append(temp_list[i]['value'].encode('utf-8'))
        except:
            Temp_conversion.append('DURATION')
            # When the type of temporal word is a period/duration, the conversion value is not present
            #eg:
                #{
                  #"end": 42,
                  #"start": 27,
                  #"text": "from 2pm to 3pm",
                  #"type": "DURATION",
                  #"value": {
                      #"begin": "T14:00",
                      #"end": "T15:00"
                  #}
    
    # Calls tempDrugOrderNegTag.py
    drug_words = tag.drug_tag(text)
    order_words = tag.order_words(text)
    neg_words = tag.neg_tag(text)

    get_index('D',' DRUG ',drug_words,text)
    get_index('O',' ORDER ',order_words,text)
    get_index('N',' NEG ',neg_words,text)
    
    # Get index for every temporal word
    for i in Temporal:
        z = re.sub(i,"TEMPORAL" ,text)
        try:
            a = nltk.word_tokenize(z)
            Temp_index.append(a.index('TEMPORAL')) 
        except: 
            z = z.replace('TEMPORAL' , ' TEMPORAL ')
            a = nltk.word_tokenize(z)
            Temp_index.append(a.index('TEMPORAL'))     
    
    
    # To sort the pattern_dict and word_dict based on indices
    od_pattern = collections.OrderedDict(sorted(pattern_dict.items()))
    od_words = collections.OrderedDict(sorted(word_dict.items()))

    indices = list(od_pattern.keys())
    pattern = list(od_pattern.values())
    words = list(od_words.values())

    return pattern, indices, words , Temporal , Temp_pattern , Temp_conversion , Temp_index

In [29]:
sentence_level = pd.read_csv('data/DF_100_users_new_tag_5-16-2018.csv')

# Take only the posts which contain treatment pattern
sentence_level = sentence_level[sentence_level['New_tag']==1]
sentence_level['Unique_id'] = sentence_level[['Post_id', 'Sent_id']].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
sentence_level = sentence_level.reset_index(drop= True)

sentence_level.head()

Post_id    User            date  forum  User_id  Sent_id  New_tag  \
0       44  Delray  2/9/2016 21:09      3     1084        2        1   
1       44  Delray  2/9/2016 21:09      3     1084        4        1   
2       44  Delray  2/9/2016 21:09      3     1084        5        1   
3       44  Delray  2/9/2016 21:09      3     1084        9        1   
4       44  Delray  2/9/2016 21:09      3     1084       12        1   

                         Preprocessed_Tokenized_Sent Unique_id  
0  my husband was diagnosed last may with melanom...      44_2  
1  because of the size and depth of the tumor mor...      44_4  
2                      5 weeks of radiation followed      44_5  
3  next week he will under go a p e t scan he had...      44_9  
4  so at this point we wait and see what the p e ...     44_12

In [30]:
# The parser takes in '%Y-%m-%d' format
sentence_level['date'] = sentence_level['date'].map(lambda x:parser.parse(x).strftime('%Y-%m-%d'))

posts_list = list(sentence_level['Preprocessed_Tokenized_Sent'])
date_list = list(sentence_level['date'])

pattern_list = []
indices_list = []
words_list = []
Temporal_list = []
Temp_pattern_list = []
Temp_conversion_list = []
Temp_index_list = []

for i in range(0,len(sentence_level['Preprocessed_Tokenized_Sent'])):
    if(i%100)==0:
        print i
    pattern, index, words, temp, temp_pattern, temp_conv, temp_ind= patternExtract(posts_list[i],date_list[i])
    
    pattern_list.append(pattern)
    indices_list.append(index)
    words_list.append(words)
    Temporal_list.append(temp)
    Temp_pattern_list.append(temp_pattern)
    Temp_conversion_list.append(temp_conv)
    Temp_index_list.append(temp_ind)
    
    
Feature_data = pd.DataFrame(
    {'Pattern': pattern_list,
     'Index': indices_list,
     'Words': words_list,
     'Temporal': Temporal_list,
     'Temp_pattern' : Temp_pattern_list,
     'Temp_conversion' : Temp_conversion_list,
     'Temp_index' : Temp_index_list,
     'Ref_Date' : date_list})
     

# Feature_data.to_csv('Pattern_out_100_user.csv' , index = False)

0
n==1 ['nivolumab', 'nivolumab', 'pembrolizumab', 'ipilimumab nivolumab']
n==1 ['ipilimumab nivolumab']
n==1 ['ipilimumab pembrolizumab']
100
200
n==1 ['nivolumab', 'ipilimumab nivolumab']
n==1 ['ipilimumab nivolumab']
n==1 ['ipilimumab vemurafenib']
300
n==1 ['carboplatin', 'temozolomide', 'imatinib', 'bevacizumab paclitaxel']
n==1 ['imatinib', 'bevacizumab paclitaxel', 'carboplatin temozolomide']
n==1 ['imatinib', 'bevacizumab paclitaxel carboplatin temozolomide']
n==1 ['vemurafenib', 'ipilimumab', 'dabrafenib mek']
n==1 ['dabrafenib mek']
400
n==1 ['vemurafenib ipilimumab']
n==1 ['cisplatin vinblastine sulfate']
500
n==1 ['bevacizumab', 'ipilimumab', 'dasatinib paclitaxel']
n==1 ['dasatinib paclitaxel', 'bevacizumab ipilimumab']
n==1 ['dabrafenib mek']
n==1 ['ipilimumab nivolumab']
n==1 ['ipilimumab nivolumab']
n==1 ['ipilimumab nivolumab']
600
n==1 ['nivolumab ipilimumab']
n==1 ['vemurafenib', 'dabrafenib mek']
n==1 ['dabrafenib mek']
n==1 ['nivolumab', 'ipilimumab nivolumab']
n==

In [33]:
Feature_data_final = pd.concat([sentence_level, Feature_data], axis=1)
Feature_data_final.head()

Post_id    User        date  forum  User_id  Sent_id  New_tag  \
0       44  Delray  2016-02-09      3     1084        2        1   
1       44  Delray  2016-02-09      3     1084        4        1   
2       44  Delray  2016-02-09      3     1084        5        1   
3       44  Delray  2016-02-09      3     1084        9        1   
4       44  Delray  2016-02-09      3     1084       12        1   

                         Preprocessed_Tokenized_Sent Unique_id Index Pattern  \
0  my husband was diagnosed last may with melanom...      44_2    []      []   
1  because of the size and depth of the tumor mor...      44_4  [47]     [N]   
2                      5 weeks of radiation followed      44_5   [4]     [O]   
3  next week he will under go a p e t scan he had...      44_9  [20]     [O]   
4  so at this point we wait and see what the p e ...     44_12    []      []   

     Ref_Date      Temp_conversion Temp_index Temp_pattern  \
0  2016-02-09            [2015-05]        [4]         [T0]   
1  2016-02-09                   []         []           []   
2  2016-02-09                [P5W]        [0]         [T0]   
3  2016-02-09  [2016-W07, 2015-SU]    [0, 14]     [T0, T1]   
4  2016-02-09                   []         []           []   

                   Temporal       Words  
0                [last may]          []  
1                        []       [not]  
2                 [5 weeks]  [followed]  
3  [next week, last summer]      [then]  
4                        []          []

In [34]:
# Convert special temporal conversions 
#Eg:
#past_pattern = 'P5W' - subtract 5 weeks
#future pattern = '2016-W08' - add 8 weeks to 1-1-2016
#date pattern = '2016-02-15' or '2016-02' or '2016' - return the same date
#date pattern also strips the extra tags like TNI, TMO, SU

past_pattern = "(P\d+(D|Y|M|W))"
future_pattern ="((?<=\s)\d{4}|^\d{4}-W\d+)"
date_pattern = '((\d{4}[/.-]\d{2}[/.-]\d{2})|(\d{4}[/.-]\d{2})|(\d{4}))'
withFew_pattern = "(PX(D|Y|M|W))"

past_reg1 = re.compile(past_pattern) 
future_reg = re.compile(future_pattern)
date_reg = re.compile(date_pattern)
withFew_reg = re.compile(withFew_pattern)

def relative_change(relative, digit=1):
    digit = int(digit)
    if relative=='D':
        return relativedelta(days=digit)
    if relative=='M':
        return relativedelta(months=digit)
    if relative=='W':
        return relativedelta(weeks=digit)
    if relative=='Y':
        return relativedelta(years=digit)

def convert_spl_temp(Feature_data):
    Drug_date = []
    for i in range(0,len(Feature_data['Temp_conversion'])):
        a = Feature_data['Temp_conversion'][i]
        matches = re.findall(past_reg1,a)
        future_match = re.findall(future_reg,a)
        ref_date = parser.parse(Feature_data['Ref_Date'])
        check_date = re.findall(date_reg, a)
        check_withfew = re.findall(withFew_reg,a)

        if a == "PRESENT_REF":
            Drug_date.append(ref_date)

        elif a == "PAST_REF":
            Drug_date.append(ref_date - relative_change('D',1))

        elif a == "FUTURE_REF":
            Drug_date.append(ref_date + relative_change('D',1))
            
        elif len(check_date)>0:
            Drug_date.append(parser.parse(check_date[0][0]))

        elif len(matches)>0:
            Drug_date.append(ref_date - relative_change(matches[0][1],filter(lambda x: x.isdigit(), matches[0][0])))

        elif len(future_match)>0:
            Drug_date.append(parser.parse(future_match[0].split('-')[0]+'-01-01') + relative_change('W',future_match[0].split('W')[1]))

        elif len(check_withfew)>0:
            continue
        # To handle exceptions    
        else :
            #The cases where we are facing exemptions during temporal words conversion (<i>"2050-01-01"</i>) 
            #are mostly duration/period tags hence exempted right now. 
            #This should be fixed in case we need to use period details also for someother use-case
            Drug_date.append(parser.parse('2050-01-01'))   
            
    return Drug_date

converted_drug_date = Feature_data_final.apply(convert_spl_temp,axis=1)

In [35]:
Feature_data_final['Drug_date']= converted_drug_date
Feature_data_final.head()

Post_id    User        date  forum  User_id  Sent_id  New_tag  \
0       44  Delray  2016-02-09      3     1084        2        1   
1       44  Delray  2016-02-09      3     1084        4        1   
2       44  Delray  2016-02-09      3     1084        5        1   
3       44  Delray  2016-02-09      3     1084        9        1   
4       44  Delray  2016-02-09      3     1084       12        1   

                         Preprocessed_Tokenized_Sent Unique_id Index Pattern  \
0  my husband was diagnosed last may with melanom...      44_2    []      []   
1  because of the size and depth of the tumor mor...      44_4  [47]     [N]   
2                      5 weeks of radiation followed      44_5   [4]     [O]   
3  next week he will under go a p e t scan he had...      44_9  [20]     [O]   
4  so at this point we wait and see what the p e ...     44_12    []      []   

     Ref_Date      Temp_conversion Temp_index Temp_pattern  \
0  2016-02-09            [2015-05]        [4]         [T0]   
1  2016-02-09                   []         []           []   
2  2016-02-09                [P5W]        [0]         [T0]   
3  2016-02-09  [2016-W07, 2015-SU]    [0, 14]     [T0, T1]   
4  2016-02-09                   []         []           []   

                   Temporal       Words  \
0                [last may]          []   
1                        []       [not]   
2                 [5 weeks]  [followed]   
3  [next week, last summer]      [then]   
4                        []          []   

                                    Drug_date  
0                       [2015-05-12 00:00:00]  
1                                          []  
2                       [2016-01-05 00:00:00]  
3  [2016-06-12 00:00:00, 2015-06-12 00:00:00]  
4                                          []

In [36]:
Feature_data_final.to_csv('All_pattern_out_100_user.csv', index=False)

# Next steps

* Fix the DURATION tags conversion 
* The extracted temporal and drugs need to be matched to get the final treatment pattern at user-level